# Aggregate Multiple Data Sources

There are several ways to integrate data sources with OpenAI and the various AI libraries. Here we're going to provide RAG over multiple data sources through plugins/tools. 
https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/using-data-retrieval-functions-for-rag


## Status
- [X] Notebook Demo
- [ ] Documentation
- [ ] App Example - csharp
- [ ] App Example - python

## Reference Documents
https://github.com/Azure/azure-cosmos-dotnet-v3


## Setup
1. Rename the [env.example.json](env.example.json) file to `env.json` and fill in the values.

In [98]:
#!import ../../config/csharp/SemanticKernelSettings.cs 
#!import ../../config/csharp/AzureAISearchSettings.cs 
#!import ../../config/csharp/CosmosSqlService.cs 
#!import code/IncidentDocument.cs 

#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.3"
#r "nuget: Azure.Search.Documents, 11.6.0"
#r "nuget: Azure.Identity, 1.12.0"
#r "nuget: Microsoft.Azure.Cosmos, 3.42.0"
#r "nuget: Microsoft.SemanticKernel, 1.18.1-rc"
#r "nuget: Microsoft.SemanticKernel.Connectors.Redis, 1.18.1-alpha"
// #r "nuget: Microsoft.SemanticKernel.Connectors.AzureCosmosDBNoSQL, 1.18.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.18.1-rc"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.18.1-preview"
#r "nuget: Microsoft.Data.Analysis, 0.21.0"
#r "nuget: System.Linq.Async, 6.0.1"
#r "nuget: CsvHelper, 33.0.1"

using System;
using System.Globalization;
using System.ComponentModel;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using System.Text.Json;
using System.Text.Json.Serialization;

using Microsoft.Data.Analysis;
using CsvHelper;
using CsvHelper.Configuration;
using Azure;
using Azure.Identity;


using Azure.Search.Documents;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes.Models;
using Azure.Search.Documents.Indexes;


using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Embeddings;
using Kernel = Microsoft.SemanticKernel.Kernel;

var (textModel, embeddingModel, openAIEndpoint, openAIKey) = SemanticKernelSettings.LoadFromFile("env.json");
var (searchEndpoint, searchKey, searchIndex) = AzureAISearchSettings.LoadFromFile("env.json");

var promptExecutionSettings = new OpenAIPromptExecutionSettings { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };

var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        endpoint: openAIEndpoint,
        apiKey: openAIKey,
        deploymentName: textModel)
    .Build();


var searchIndexClient = new SearchClient(new Uri(searchEndpoint), searchIndex, new AzureKeyCredential(searchKey));
var cosmosService = CosmosSqlService.CreateFromEnvFile("env.json");

var loadIncidentData = false;


Installed Packages Azure.AI.OpenAI, 2.0.0-beta.3 Azure.Identity, 1.12.0 Azure.Search.Documents, 11.6.0 CsvHelper, 33.0.1 Microsoft.Azure.Cosmos, 3.42.0 Microsoft.Data.Analysis, 0.21.0 Microsoft.SemanticKernel, 1.18.1-rc Microsoft.SemanticKernel.Connectors.OpenAI, 1.18.1-rc Microsoft.SemanticKernel.Connectors.Redis, 1.18.1-alpha Microsoft.SemanticKernel.Planners.OpenAI, 1.18.1-preview System.Linq.Async, 6.0.1

In [99]:

if(loadIncidentData)
{
    using(var reader = new StreamReader("data/incidents.csv"))
    using(var csv = new CsvReader(
        reader: reader, 
        configuration: new CsvConfiguration(CultureInfo.InvariantCulture)
        {
            // The document model has an Id property that is set for Cosmos
            HeaderValidated = null,
            MissingFieldFound = null,
        }))    
    await foreach (var record in csv.GetRecordsAsync<IncidentDocument>())
    {
        ItemResponse<IncidentDocument> itemResponse = null;
        try
        {
            itemResponse = await cosmosService.containerClient.CreateItemAsync(record);
        }
        catch (Exception _)
        {
            Console.WriteLine($"Exception {JsonSerializer.Serialize(itemResponse)}.");
        }
    }
}

In [ ]:
public interface IPlugin
{
    string Name { get; }
}

public class TriagePlugin : IPlugin
{
    public string Name => "TriagePlugin";

    public async Task<IPluginResponse> ExecuteAsync(IPluginRequest request, IPluginResponse response)
    {
        var query = request.Query;
        var result = await semanticKernel.ExecuteAsync(query);
        return response with { Result = result };
    }
}

In [94]:

// Optionally create the Azure Search Index

// public class IncidentIndexRecord
// {
//     public static SearchIndex GetIndex(Configuration configuration)
//     {
//         const string vectorSearchHnswProfile = "hnsw-vector-profile";
//         const string vectorSearchExhasutiveKnnProfile = "myExhaustiveKnnProfile";
//         const string vectorSearchHnswConfig = "myHnsw";
//         const string vectorSearchExhaustiveKnnConfig = "myExhaustiveKnn";
//         const string vectorSearchVectorizer = "myOpenAIVectorizer";
//         const string semanticSearchConfig = "my-semantic-config";
//         const int modelDimensions = 1536;

//         SearchIndex searchIndex = new(configuration.IndexName)
//         {
//             VectorSearch = new()
//             {
//                 Profiles =
//                 {
//                     new VectorSearchProfile(vectorSearchHnswProfile, vectorSearchHnswConfig)
//                     {
//                         Vectorizer = vectorSearchVectorizer
//                     },
//                     new VectorSearchProfile(vectorSearchExhasutiveKnnProfile, vectorSearchExhaustiveKnnConfig)
//                 },
//                 Algorithms =
//                 {
//                     new HnswAlgorithmConfiguration(vectorSearchHnswConfig),
//                     new ExhaustiveKnnAlgorithmConfiguration(vectorSearchExhaustiveKnnConfig)
//                 },
//                 Vectorizers =
//                 {
//                     new AzureOpenAIVectorizer(vectorSearchVectorizer)
//                     {
//                         AzureOpenAIParameters = new AzureOpenAIParameters()
//                         {
//                             ResourceUri = new Uri(configuration.AzureOpenAIEndpoint),
//                             ApiKey = configuration.AzureOpenAIApiKey,
//                             DeploymentId = configuration.AzureOpenAIEmbeddingDeployedModel,
//                         }
//                     }
//                 }
//             },
//             SemanticSearch = new()
//             {
//                 Configurations =
//                 {
//                     new SemanticConfiguration(semanticSearchConfig, new()
//                     {
//                         TitleField = new SemanticField(fieldName: "title"),
//                         ContentFields =
//                         {
//                             new SemanticField(fieldName: "chunk")
//                         },
//                     })
//                 },
//             },
//             Fields =
//             {
//                 new SearchableField("parent_id") { IsFilterable = true, IsSortable = true, IsFacetable = true },
//                 new SearchableField("chunk_id") { IsKey = true, IsFilterable = true, IsSortable = true, IsFacetable = true, AnalyzerName = LexicalAnalyzerName.Keyword },
//                 new SearchableField("title"),
//                 new SearchableField("chunk"),
//                 new SearchField("vector", SearchFieldDataType.Collection(SearchFieldDataType.Single))
//                 {
//                     IsSearchable = true,
//                     VectorSearchDimensions = modelDimensions,
//                     VectorSearchProfileName = vectorSearchHnswProfile
//                 },
//                 new SearchableField("category") { IsFilterable = true, IsSortable = true, IsFacetable = true },
//             },
//         };

//         return searchIndex;
//     }
// }

Error: (5,40): error CS0246: The type or namespace name 'Configuration' could not be found (are you missing a using directive or an assembly reference?)

### Semantic Search

In [76]:
// Disabling Experimental warnings
#pragma warning disable SKEXP0001

public class SemanticSearchPlugin
{
    private readonly ITextEmbeddingGenerationService _textEmbeddingGenerationService;
    private readonly SearchIndexClient _indexClient;

    public SemanticSearchPlugin(ITextEmbeddingGenerationService textEmbeddingGenerationService, SearchIndexClient indexClient)
    {
        _textEmbeddingGenerationService = textEmbeddingGenerationService;
        _indexClient = indexClient;
    }

    [KernelFunction("Search")]
    [Description("Search for a document similar to the given query.")]
    public async Task<string> SearchAsync(string query)
    {
        // Convert string query to vector
        ReadOnlyMemory<float> embedding = await _textEmbeddingGenerationService.GenerateEmbeddingAsync(query);

        // Get client for search operations
        SearchClient searchClient = _indexClient.GetSearchClient("default-collection");

        // Configure request parameters
        VectorizedQuery vectorQuery = new(embedding);
        vectorQuery.Fields.Add("vector");

        SearchOptions searchOptions = new() { VectorSearch = new() { Queries = { vectorQuery } } };

        // Perform search request
        var response = await searchClient.SearchAsync<IndexSchema>(searchOptions);

        // Collect search results
        await foreach (SearchResult<IndexSchema> result in response.Value.GetResultsAsync())
        {
            return result.Document.Chunk; // Return text from first result
        }

        return string.Empty;
    }

    private sealed class IndexSchema
    {
        [JsonPropertyName("chunk")]
        public string Chunk { get; set; }

        [JsonPropertyName("vector")]
        public ReadOnlyMemory<float> Vector { get; set; }
    }
}

### Classic Search
- NL2SQL

In [ ]:
public class ClassicSearchPlugin
{
    private readonly Microsoft.Azure.Cosmos.Container _cosmosContainer;

    public ClassicSearchPlugin(Microsoft.Azure.Cosmos.Container cosmosContainer)
    {
        _cosmosContainer = cosmosContainer;
    }

    [KernelFunction("GetCustomerInfo")]
    [Description("Retrieve customer information based on the given customer ID.")]
    public async Task<Customer> GetCustomerInfoAsync(string customerId)
    {
        return await _cosmosContainer.GetCustomerInfoAsync(customerId);
    }
}